### GAN

In [2]:
class GAN():
    def __init__(self):
        self.batch_size = None
        self.timestep = xtrain.shape[1]
        self.feature = xtrain.shape[2]
        optimizer = Adam(0.01,0.99)
        self.landa1 = 1
        self.landa2 = 1
        
        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(optimizer = optimizer , loss = 'binary_crossentropy')
        # Build the generator
        self.generator = self.build_generator()
        # The generator takes real data as input and generates data
        gen_output = self.generator(self.inputs)
        # The discriminator takes generated data as input and determines dis_output
        concat = concatenate([self.inputs,gen_output], axis = 1)
        dis_input = Reshape(((self.timestep+1)*self.feature,))(concat)
        dis_output = self.discriminator(dis_input)
        # The combined model  (stacked generator and discriminator)
        self.discriminator.trainable = False
        
        self.combined = Model(self.inputs, dis_output)
        self.combined.compile(optimizer = optimizer, loss = 'binary_crossentropy')
        
        
    def build_generator(self):
        self.inputs = Input(batch_shape = (self.batch_size, self.timestep, self.feature))
        lstm1 = LSTM(256, return_sequences = True, stateful = False)(self.inputs)
        batch1 = BatchNormalization()(lstm1)
        drop1 = Dropout(0.2)(batch1)
        lstm2 = LSTM(128, return_sequences = True, stateful = False)(drop1)
        batch2 = BatchNormalization()(lstm2)
        drop2 = Dropout(0.2)(batch2)
        lstm3 = LSTM(64, return_sequences = True, stateful = False)(drop2)
        batch3 = BatchNormalization()(lstm3)
        drop3 = Dropout(0.2)(batch3)
        lstm4 = LSTM(1, return_sequences = True, stateful = False)(drop3)
        batch4 = BatchNormalization()(lstm4)
        drop4 = Dropout(0.2)(batch4)
        fl = Flatten()(drop4)
        re = Reshape((1,22))(fl)
        rp = Dense(19)(re)
        return Model(self.inputs,rp)
   

    def build_discriminator(self):
        model = Sequential()
        model.add(Dense(units = 128, activation = 'relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.3))
        model.add(Dense(units = 64, activation = 'relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.3))
        model.add(Dense(units = 32, activation = 'relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.3))
        
        model.add(Dense(units = 1, activation = 'relu'))
        d_input = Input(batch_shape = (self.batch_size, ((self.timestep+1)*self.feature)))
        d_output = model(d_input)
        return Model(d_input, d_output)
    
    def train(self, epochs, batch_size):
        self.D_loss = []
        self.g_loss = []
        self.g_MSE = []
        self.G_loss= []
        for epoch in range(epochs):
            for i in range(0,xtrain.shape[0]//batch_size):
                x_train_batch = xtrain[batch_size*i:batch_size*i+batch_size,:,:]
                y_train_batch = ytrain[batch_size*i:batch_size*i+batch_size,:,:]
                y_pred_batch = self.generator.predict(x_train_batch)
                
                
                real = np.concatenate((x_train_batch,y_train_batch), axis=1)
                fake = np.concatenate((x_train_batch,y_pred_batch), axis=1)
                label_real = np.ones((real.shape[0],1))
                label_fake = np.zeros((fake.shape[0],1))
                real = np.reshape(real, (batch_size, (self.timestep+1)*self.feature))
                fake = np.reshape(fake, (batch_size, (self.timestep+1)*self.feature))
                
                # Train the discriminator
#                 self.discriminator.trainable = True
                
                # Train_on_batch saves the previous weights for each i iteration
                d_loss_real = self.discriminator.train_on_batch(real, label_real) 
                d_loss_fake = self.discriminator.train_on_batch(fake, label_fake)
                d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
                self.D_loss.append(d_loss)
                D_loss = np.array(self.D_loss)
                
                
                # Train the generator (to have the discriminator label samples as label_real)
                # For the combined model we will only train the generator
                
                gan_loss = self.combined.train_on_batch(x_train_batch, label_fake)
                x_pred_g = self.generator.predict(x_train_batch)
                m = xtrain.shape[0]//batch_size
                self.g_MSE.append(np.sum((1/m)*(x_pred_g[:,0,4:5]-y_train_batch[:,0,4:5])**2))
                self.g_loss.append(gan_loss)
                self.g_MSE1 = np.array(self.g_MSE)
                self.g_loss1 = np.array(self.g_loss)
                self.G_loss = np.add(self.landa1*self.g_loss1,self.landa2*self.g_MSE1)
                # Print the Results
                print ("epoch : %d , batch : %d , GAN loss : %f" % (epoch, i, gan_loss))
            self.combined.reset_states()
        
        
    def predict(self):
        self.predicted = self.generator.predict(xtest)
        self.predicted = np.array(self.predicted)
        
        
    def plot(self):
        plt.figure()
        plt.plot(self.D_loss, color = 'blue')
        plt.ylabel("Discriminator Loss", fontsize=13)
        plt.xlabel('Iterations of Batch', fontsize=13)
        fig1 = plt.gcf()
        fig1.savefig('diss_loss.png', dpi=500, bbox_inches='tight')
        plt.show()
        
        plt.figure()
        plt.plot(self.G_loss, color = 'red')
        plt.ylabel("Generator Loss", fontsize=13)
        plt.xlabel('Iterations of Batch', fontsize=13)
        fig2 = plt.gcf()
        fig2.savefig('gen_loss.png', dpi=500, bbox_inches='tight')
        plt.show()
    
if __name__ == '__main__':
    gan = GAN()
    gan.train(epochs = 5, batch_size = 32)
    gan.predict()
    gan.plot()
